In [1]:
import asyncio
import numpy as np
from temporalio.client import Client
from datetime import datetime
import os

# Import the workflow and related classes from previous implementation
from inference_workflow import (
    AudioChunk,
    AudioClassificationWorkflow,
    run_worker,
    start_classification
)


/home/sirjanh/miniconda3/envs/FrontERAAssignment/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-02-23 19:09:38.042278: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-23 19:09:38.151609: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-02-23 19:09:38.151628: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on y

In [2]:
async def main():
    
    # 1. Start the Temporal server (make sure it's running)
    # You should have already installed and started the Temporal server
    # typically running on localhost:7233
    
    # 2. Create sample audio chunks for testing
    sample_chunks = []
    for i in range(3):  # Create 3 test chunks
        # Create synthetic audio data (replace with your real audio data)
        duration = 10  # 10 seconds
        sample_rate = 16000  # 16kHz
        
        # Create synthetic audio data (replace this with your real audio loading)
        audio_data = np.random.randn(int(duration * sample_rate))
        
        chunk = AudioChunk(
            data=audio_data,
            timestamp=datetime.now().timestamp(),
            sample_rate=sample_rate,
            chunk_id=f"chunk_{i}",
            duration=duration,
            metadata={"source": "test", "sequence": i}
        )
        sample_chunks.append(chunk)
    
    # 3. Start worker in the background
    worker_task = asyncio.create_task(run_worker())
    
    # 4. Wait a bit for worker to initialize
    await asyncio.sleep(2)
    
    try:
        # 5. Run the workflow
        print("Starting classification workflow...")
        results = await start_classification(sample_chunks)
        
        # 6. Print results
        print("\nClassification Results:")
        for result in results:
            print(f"\nChunk ID: {result['chunk_id']}")
            print(f"Predictions: {result['predictions']}")
            print(f"Confidence: {result['confidence']}")
            
    except Exception as e:
        print(f"Error running workflow: {e}")
    finally:
        # 7. Cancel worker
        worker_task.cancel()
        try:
            await worker_task
        except asyncio.CancelledError:
            pass


In [3]:
asyncio.run(main())

RuntimeError: asyncio.run() cannot be called from a running event loop